In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/stream_e.csv", sep = "ef5r64i", engine = "python")
print(df.columns, df.shape)


Index(['Texto', 'IdUsuario', 'FechaCreacion', 'IdTweet', 'NombreUsuario',
       'FechaCreacionTweet', 'Hashtags', 'Url'],
      dtype='object') (1268, 8)


In [3]:
#Creating the dict with unique users and a list for the tweets they have written
corpus = {}
for i in range(df.shape[0]):
    if df.iloc[i]["NombreUsuario"] in corpus:
        corpus[df.iloc[i]["NombreUsuario"]].append(df.iloc[i]["Texto"])
    else:
        corpus[df.iloc[i]["NombreUsuario"]] = [df.iloc[i]["Texto"]]
type(corpus), len(corpus)

(dict, 1189)

In [4]:
documents = list(df.Texto)

In [5]:
#LSA

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
  
# raw documents to tf-idf matrix: 
vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)
# SVD to reduce dimensionality: 
svd_model = TruncatedSVD(n_components=100,         # num dimensions
                         algorithm='randomized',
                         n_iter=10)
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])
svd_matrix = svd_transformer.fit_transform(documents)

# svd_matrix can later be used to compare documents, compare words, or compare queries with documents

In [6]:
#LDA

import gensim

from gensim.corpora.Dictionary import load_from_text, doc2bow
from gensim.corpora import MmCorpus
from gensim.models.ldamodel import LdaModel
document = "This is some document..."
# load id->word mapping (the dictionary)
id2word = load_from_text('wiki_en_wordids.txt')
# load corpus iterator
mm = MmCorpus('wiki_en_tfidf.mm')
# extract 100 LDA topics, updating once every 10,000
lda = LdaModel(corpus=mm, id2word=id2word, num_topics=100, update_every=1, chunksize=10000, passes=1)
# use LDA model: transform new doc to bag-of-words, then apply lda
doc_bow = doc2bow(document.split())
doc_lda = lda[doc_bow]
# doc_lda is vector of length num_topics representing weighted presence of each topic in the doc

ModuleNotFoundError: No module named 'gensim.corpora.Dictionary'

---------

## generando un txt por usuario con los tweets

In [ ]:
import pandas as pd
import pyodbc 
import configLocal

server = configLocal.server
database = configLocal.database
username = configLocal.username
password = configLocal.password
cnxn = configLocal.cnxn
cursor = cnxn.cursor()

In [ ]:
data = pd.read_sql_query("select msg_id, msg_users_id, msg_timestamp, msg_text from dbo.msg", cnxn)

In [ ]:
data.head()

In [ ]:
corpus = {}
for i in range(data.shape[0]):
    if data.iloc[i]["msg_users_id"] in corpus:
        corpus[data.iloc[i]["msg_users_id"]].append(data.iloc[i]["msg_text"])
    else:
        corpus[data.iloc[i]["msg_users_id"]] = [data.iloc[i]["msg_text"]]
type(corpus), len(corpus)

In [ ]:
type(corpus["428308122"])

In [ ]:
a = corpus["428308122"]

In [ ]:
for i in corpus:
    with open("C:/Users/alonsoga/OneDrive - HP Inc/Oficina/Documentacion/DS/RoundU/data/user_twit/{}.txt".format(i), "x") as file:
        file.write(" ".join(corpus[i]))

---------

# eXAMPLE 1

In [10]:
import pandas as pd
import pyodbc 
import configLocal

server = configLocal.server
database = configLocal.database
username = configLocal.username
password = configLocal.password
cnxn = configLocal.cnxn
cursor = cnxn.cursor()

In [11]:
data = pd.read_sql_query("select msg_id, msg_users_id, msg_timestamp, msg_text from dbo.msg", cnxn)

In [12]:
corpus = []

for i in range(len(data)):
    corpus.append(data.iloc[i]["msg_text"])

In [13]:
import gensim
import logging
import tempfile
import os

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

from gensim import corpora
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Folder "C:\Users\alonsoga\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


In [14]:
from nltk.corpus import stopwords
from string import punctuation

# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('spanish') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

In [15]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'elon.dict'))  # store the dictionary, for future reference
print(dictionary)

2019-05-13 10:22:48,477 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-05-13 10:22:48,680 : INFO : adding document #10000 to Dictionary(41352 unique tokens: ['@aritzeta', '@bibiloni', '@jbadia16', '@josepmcld', '@ratafia_party']...)
2019-05-13 10:22:48,883 : INFO : adding document #20000 to Dictionary(71659 unique tokens: ['@aritzeta', '@bibiloni', '@jbadia16', '@josepmcld', '@ratafia_party']...)
2019-05-13 10:22:48,962 : INFO : built Dictionary(81793 unique tokens: ['@aritzeta', '@bibiloni', '@jbadia16', '@josepmcld', '@ratafia_party']...) from 23896 documents (total 241199 corpus positions)
2019-05-13 10:22:48,962 : INFO : saving Dictionary object under C:\Users\alonsoga\AppData\Local\Temp\elon.dict, separately None
2019-05-13 10:22:48,963 : WARNING : this function is deprecated, use smart_open.open instead
2019-05-13 10:22:49,001 : INFO : saved C:\Users\alonsoga\AppData\Local\Temp\elon.dict


Dictionary(81793 unique tokens: ['@aritzeta', '@bibiloni', '@jbadia16', '@josepmcld', '@ratafia_party']...)


In [16]:
print(dictionary.token2id)

{'@aritzeta': 0, '@bibiloni': 1, '@jbadia16': 2, '@josepmcld': 3, '@ratafia_party': 4, 'diria': 5, 'https://t.co/25w8rz6vlq': 6, 'https://t.co/uhxfhidu0n': 7, 'jo': 8, 'però': 9, 'reflexiu': 10, 'sobrer:': 11, 'també.': 12, 'és': 13, 'fuerte!!!': 14, 'lastima': 15, '@helga_tito': 16, 'bro': 17, 'casters': 18, 'comentaba.': 19, 'lvp.': 20, 'madre': 21, 'mejores': 22, 'parece': 23, 'puta': 24, 'quejo,': 25, 'si': 26, 'solo': 27, '@iescolar': 28, 'evidente!!!': 29, '??': 30, 'días': 31, 'karen': 32, 'manda': 33, 'msj': 34, '40%': 35, '??los': 36, '@autonomosata': 37, '@lorenzoamor_ata': 38, 'alertan': 39, 'autónomos': 40, 'año': 41, 'creación': 42, 'desplomará': 43, 'empleo': 44, 'https://t.co/gixdu7usml': 45, 'https://t.co/mzzdyw28ap': 46, '@supersonia99': 47, 'jajaja': 48, 'madurao': 49, ':(': 50, 'quiero': 51, '@gobysimon': 52, 'ahaha': 53, 'currently': 54, 'doing': 55, 'get': 56, 'i’m': 57, 'perfectly': 58, 'same.': 59, 'the': 60, 'you': 61, '@conchetta9': 62, '@educitof': 63, 'engañó

In [17]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)  # store to disk, for later use

2019-05-13 10:22:49,557 : INFO : storing corpus in Matrix Market format to C:\Users\alonsoga\AppData\Local\Temp\elon.mm
2019-05-13 10:22:49,557 : WARNING : this function is deprecated, use smart_open.open instead
2019-05-13 10:22:49,560 : INFO : saving sparse matrix to C:\Users\alonsoga\AppData\Local\Temp\elon.mm
2019-05-13 10:22:49,561 : INFO : PROGRESS: saving document #0
2019-05-13 10:22:49,585 : INFO : PROGRESS: saving document #1000
2019-05-13 10:22:49,607 : INFO : PROGRESS: saving document #2000
2019-05-13 10:22:49,628 : INFO : PROGRESS: saving document #3000
2019-05-13 10:22:49,649 : INFO : PROGRESS: saving document #4000
2019-05-13 10:22:49,671 : INFO : PROGRESS: saving document #5000
2019-05-13 10:22:49,691 : INFO : PROGRESS: saving document #6000
2019-05-13 10:22:49,713 : INFO : PROGRESS: saving document #7000
2019-05-13 10:22:49,738 : INFO : PROGRESS: saving document #8000
2019-05-13 10:22:49,759 : INFO : PROGRESS: saving document #9000
2019-05-13 10:22:49,785 : INFO : PROGR

In [18]:
from gensim import corpora, models, similarities


In [19]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model


2019-05-13 10:22:50,128 : INFO : collecting document frequencies
2019-05-13 10:22:50,129 : INFO : PROGRESS: processing document #0
2019-05-13 10:22:50,154 : INFO : PROGRESS: processing document #10000
2019-05-13 10:22:50,182 : INFO : PROGRESS: processing document #20000
2019-05-13 10:22:50,195 : INFO : calculating IDF weights for 23896 documents and 81792 features (236189 matrix non-zeros)


In [20]:
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors


LDA

In [21]:
total_topics = 10


In [22]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2019-05-13 10:23:17,806 : INFO : using symmetric alpha at 0.1
2019-05-13 10:23:17,807 : INFO : using symmetric eta at 0.1
2019-05-13 10:23:17,821 : INFO : using serial LDA version on this node
2019-05-13 10:23:17,913 : INFO : running online (single-pass) LDA training, 10 topics, 1 passes over the supplied corpus of 23896 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-05-13 10:23:17,914 : INFO : PROGRESS: pass 0, at document #2000/23896
2019-05-13 10:23:19,772 : INFO : merging changes from 2000 documents into a model of 23896 documents
2019-05-13 10:23:19,824 : INFO : topic #5 (0.100): 0.009*"si" + 0.008*"trancar" + 0.008*"ser" + 0.006*"??" + 0.005*"asi" + 0.004*"enfrenten" + 0.004*"toda" + 0.004*"vuelve" + 0.004*"romperles" + 0.004*"huevos"
2019-05-13 10:23:19,826 : INFO : topic #8 (0.100): 0.004*"si" + 0.003*"ser" + 0.003*"gente" + 0.002*"ninguna" + 0.002*"??" + 0.002*"????" 

2019-05-13 10:23:25,269 : INFO : topic #3 (0.100): 0.009*"messi" + 0.008*"si" + 0.007*"menos" + 0.007*"peor" + 0.005*"buenas" + 0.005*"sí." + 0.004*"vi" + 0.004*"nombre" + 0.004*"jordi" + 0.004*"equipo"
2019-05-13 10:23:25,271 : INFO : topic #6 (0.100): 0.011*"sistema" + 0.009*"nivel" + 0.009*"barcelona" + 0.009*"hace" + 0.008*"gusta" + 0.007*"si" + 0.007*"#dgt" + 0.007*"(pk" + 0.007*"#retención" + 0.007*"#dgtbarcelona"
2019-05-13 10:23:25,273 : INFO : topic #5 (0.100): 0.021*"si" + 0.013*"ser" + 0.005*"todas" + 0.005*"tal" + 0.005*"????????" + 0.005*"toda" + 0.004*"tras" + 0.004*"ahora" + 0.004*"puede" + 0.004*"dia"
2019-05-13 10:23:25,274 : INFO : topic #0 (0.100): 0.008*"#barcelona" + 0.006*"cada" + 0.006*"vez" + 0.005*"cualquier" + 0.005*"queda" + 0.005*"sí," + 0.005*"si" + 0.004*"muchas" + 0.004*"mano" + 0.004*"sigo"
2019-05-13 10:23:25,276 : INFO : topic #8 (0.100): 0.006*"puto" + 0.006*"fotos" + 0.006*"ninguna" + 0.006*"i" + 0.006*"pasa" + 0.005*"&amp;" + 0.005*"sólo" + 0.004*"e

In [23]:
#Show first n important word in the topics:
lda.show_topics(total_topics,10)

[(0,
  '0.006*"queda" + 0.005*"#barcelona" + 0.005*"cada" + 0.005*"momento" + 0.005*"cualquier" + 0.005*"sí," + 0.004*"twitter" + 0.004*"vez" + 0.004*"muchas" + 0.004*"hago"'),
 (1,
  '0.018*"k" + 0.008*"tan" + 0.007*"siempre" + 0.006*"luego" + 0.005*"grande" + 0.005*"ve" + 0.005*"españa" + 0.005*"mejor" + 0.004*"bien" + 0.004*"casi"'),
 (2,
  '0.012*"??" + 0.006*"???" + 0.005*"pa" + 0.005*"madrid" + 0.005*"#26m" + 0.004*"escribe" + 0.004*"quiere" + 0.004*"amigo" + 0.004*"plan" + 0.003*"viendo"'),
 (3,
  '0.010*"buenas" + 0.009*"peor" + 0.007*"forma" + 0.007*"noches" + 0.006*"domingo" + 0.005*"messi" + 0.005*"menos" + 0.005*"vi" + 0.005*"si" + 0.004*"debería"'),
 (4,
  '0.024*"q" + 0.013*"si" + 0.008*"barcelona," + 0.008*"gracias" + 0.007*"pues" + 0.006*"spain" + 0.006*"así" + 0.006*"espero" + 0.005*"d" + 0.005*"fin"'),
 (5,
  '0.021*"si" + 0.015*"ser" + 0.007*"tras" + 0.006*"tal" + 0.006*"????????" + 0.005*"quieres" + 0.005*"vía" + 0.005*"dia" + 0.004*"todas" + 0.004*"manera"'),
 (6,


In [24]:
from collections import OrderedDict

data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
#data_lda

In [25]:
import pandas as pd

df_lda = pd.DataFrame(data_lda)
print(df_lda.shape)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

(217, 10)
(10, 217)


In [26]:
df_lda

,"""la",#26m,#barcelona,#rcde,#valverdeout,&amp;,...,....,15,3,...,viendo,viva,vivir,voy,vía,x,xd,¡qué,¿qué,última
0,0.000000,0.000000,0.005407,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.004808,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003398,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.002696,0.000000,0.000000
3,0.003191,0.000000,0.000000,0.004175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.004706,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.005341,0.000000,0.000000,0.000000,0.003361,0.006034,...,0.000000,0.00298,0.003881,0.00000,0.000000,0.004334,0.000000,0.000000,0.000000,0.003389
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00521,0.000000,0.000000,0.004986,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.003373,0.000000,0.003409,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.003383,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003734,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [27]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
5      -33.544483 -124.577492       1        1  26.820568
7      102.147461  -89.762520       2        1  10.307327
2      172.141434  -47.219048       3        1   7.965099
8      -31.140217  -20.576704       4        1   7.843866
3       76.347931  -17.865620       5        1   7.843865
0       27.077391  -76.050339       6        1   7.843860
6       29.928989   46.205322       7        1   7.843857
1       54.707581 -162.257950       8        1   7.843857
9      133.532288   36.097847       9        1   7.843852
4      152.571182 -148.464005      10        1   7.843848, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
30    Default  176.000000          ??  176.000000  30.0000  30.0000
115   Default   48.000000           q   48.000000  29.0000  29.0000
1509  Default   33.000000           k   33.000000  28.0000  28.0000
668   Default   63.000000        ????   63.000000  27.0000  27.0000
192   Default  123.000000         ser  123.000000  26.0000  26.0000
7810  Default   44.000000        tras   44.000000  25.0000  25.0000
722   Default   44.000000         hoy   44.000000  24.0000  24.0000
2126  Default   19.000000      buenas   19.000000  23.0000  23.0000
1477  Default   17.000000        peor   17.000000  22.0000  22.0000
1058  Default   39.000000         tal   39.000000  21.0000  21.0000
171   Default   55.000000         ver   55.000000  20.0000  20.0000
1220  Default   18.000000        puto   18.000000  19.0000  19.0000
26    Default  239.000000          si  239.000000  18.0000  18.0000
1703  Default   33.000000         tan   33.000000  17.0000  17.0000
133   Default   32.000000     quieres   32.000000  16.0000  16.0000
4791  Default   14.000000    entiendo   14.000000  15.0000  15.0000
188   Default   43.000000        hace   43.000000  14.0000  14.0000
164   Default   56.000000     gracias   56.000000  13.0000  13.0000
651   Default   15.000000  barcelona,   15.000000  12.0000  12.0000
372   Default   48.000000       ahora   48.000000  11.0000  11.0000
922   Default   17.000000       gusta   17.000000  10.0000  10.0000
103   Default   37.000000     siempre   37.000000   9.0000   9.0000
51    Default   27.000000      quiero   27.000000   8.0000   8.0000
4020  Default   29.000000         dia   29.000000   7.0000   7.0000
563   Default   17.000000       forma   17.000000   6.0000   6.0000
753   Default   30.000000   barcelona   30.000000   5.0000   5.0000
4394  Default   12.000000      noches   12.000000   4.0000   4.0000
3468  Default   27.000000         mal   27.000000   3.0000   3.0000
2867  Default   15.000000       feliz   15.000000   2.0000   2.0000
420   Default   42.000000        pues   42.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
587   Topic10    5.063179         iba    6.355371   2.3181  -5.9140
789   Topic10    9.470858           d   14.271137   2.1354  -5.2878
354   Topic10    4.407400      entrar    5.544721   2.3159  -6.0527
2052  Topic10    2.938153      menudo    3.231962   2.4501  -6.4582
3868  Topic10    3.364763    publicar    3.932054   2.3896  -6.3227
3214  Topic10    4.111643       habla    5.342867   2.2835  -6.1222
315   Topic10    3.492631      centro    4.205741   2.3596  -6.2854
1189  Topic10    6.052833        foto   10.326759   2.0112  -5.7355
1100  Topic10   11.417560         así   29.001030   1.6133  -5.1009
3217  Topic10    3.807054         mil    4.956406   2.2816  -6.1992
420   Topic10   12.843165        pues   42.480637   1.3492  -4.9832
1439  Topic10    6.755048      semana   14.027124   1.8147  -5.6257
164   Topic10   14.135108     gracias   56.309780   1.1632  -4.8874
3652  Topic10    4.073215   realmente    5.735470   2.2032  -6.1316
501   Topic10    6.943996       decir   17.191986   1.6389  -5.59

---------

extract entities: https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [ ]:
ex = 'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'

In [ ]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent, language="spanish")
    sent = nltk.pos_tag(sent, lang="spanish")
    return sent

In [ ]:
sent = preprocess(ex)
sent

In [ ]:
ex = str([word for word in corpus])

In [ ]:
preprocess(corpus[0])

https://stackoverflow.com/questions/14732465/nltk-tagging-spanish-words-using-a-corpus  
    https://www.quora.com/Which-languages-are-available-in-nltk-part-of-speech-tagger

-----------


In [ ]:
import nltk

In [ ]:
nltk.download()

In [ ]:
from nltk.corpus import gutenberg

In [ ]:
print (gutenberg.fileids())

In [ ]:
from nltk.corpus import gutenberg
for fileid in gutenberg.fileids():
    num_chars = len(gutenberg.raw(fileid))
    num_tokens = len(gutenberg.words(fileid))
    num_sents = len(gutenberg.sents(fileid))
    num_vocab = len(set(w.lower() for w in gutenberg.words(fileid)))
    print (str(num_chars) +  "-" + str(num_tokens) + "-" + str(num_sents) +  "-" + str(num_vocab))

In [ ]:
from nltk.corpus import PlaintextCorpusReader
import nltk
wordlists = PlaintextCorpusReader("C:/Users/alonsoga/OneDrive - HP Inc/Oficina/Documentacion/DS/RoundU/data/user_twit" ,".*\.txt", 
                                  encoding="Latin1")
#wordlists.words('stream_e.csv')
print ("Sentences:" +str(len(wordlists.sents())))
for sentence in wordlists.sents():
    tokens = nltk.word_tokenize(str(sentence))
    tagged_tokens = nltk.pos_tag(tokens)
    verbs = 0
    for tagged in tagged_tokens:
        token, tag = tagged
        if tag == 'VBP' or tag == 'VB':
            verbs += 1
    print ("Tokens:" +str(len(tokens)) +  "-" +"Verbs:"+str(verbs))

--------------

# tutorial lda

https://nlpforhackers.io/topic-modeling/

In [ ]:
from nltk.corpus import brown
 
data = []
 
for fileid in brown.fileids():
    document = ' '.join(brown.words(fileid))
    data.append(document)
 
NO_DOCUMENTS = len(data)
print(NO_DOCUMENTS)
print(data[:5])

In [ ]:
brown.fileids()

In [ ]:
import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
 
NUM_TOPICS = 10
STOPWORDS = stopwords.words('english')
 
def clean_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text
 
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in data:
    tokenized_data.append(clean_text(text))
 
 
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...
 
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 

In [ ]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)
 

In [ ]:
text = "The economy is working better than ever"
bow = dictionary.doc2bow(clean_text(text))
 
print(lsi_model[bow])
# [(0, 0.091615426138426506), (1, -0.0085557463300508351), (2, 0.016744863677828108), (3, 0.040508186718598529), (4, 0.014201267714185898), (5, -0.012208538275305329), (6, 0.031254053085582149), (7, 0.017529584659403553), (8, 0.056957633371540077),(9, 0.025989149894888153)]
 
print(lda_model[bow])
# [(0, 0.020005183), (1, 0.020005869), (2, 0.02000626), (3, 0.020005472), (4, 0.020009108), (5, 0.020005926), (6, 0.81994385), (7, 0.020006068), (8, 0.020006327), (9, 0.020005994)]


In [ ]:
from gensim import similarities
 
lda_index = similarities.MatrixSimilarity(lda_model[corpus])
 
# Let's perform some queries
similarities = lda_index[lda_model[bow]]
# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda item: -item[1])
 
# Top most similar documents:
print(similarities[:10])
# [(104, 0.87591344), (178, 0.86124849), (31, 0.8604598), (77, 0.84932965), (85, 0.84843522), (135, 0.84421808), (215, 0.84184396), (353, 0.84038532), (254, 0.83498049), (13, 0.82832891)]
 
# Let's see what's the most similar document
document_id, similarity = similarities[0]
print(data[document_id][:1000])

------

In [ ]:
texto = []
for i in corpus:
    texto.append(corpus[i])

In [ ]:
texto[0]

In [ ]:
data = []

for fileid in texto:
    document = ' '.join(fileid)
    data.append(document)
 
NO_DOCUMENTS = len(data)
print(NO_DOCUMENTS)
print(dato[:5])

In [ ]:
import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
 
NUM_TOPICS = 10
STOPWORDS = stopwords.words('spanish')

In [ ]:
def clean_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text
 
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in dato:
    tokenized_data.append(clean_text(text))
 
 
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...
 
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 

In [ ]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)
 

In [ ]:
text = "la economia funciona mejor que nunca"
bow = dictionary.doc2bow(clean_text(text))
 
print(lsi_model[bow])
# [(0, 0.091615426138426506), (1, -0.0085557463300508351), (2, 0.016744863677828108), (3, 0.040508186718598529), (4, 0.014201267714185898), (5, -0.012208538275305329), (6, 0.031254053085582149), (7, 0.017529584659403553), (8, 0.056957633371540077),(9, 0.025989149894888153)]
 
print(lda_model[bow])
# [(0, 0.020005183), (1, 0.020005869), (2, 0.02000626), (3, 0.020005472), (4, 0.020009108), (5, 0.020005926), (6, 0.81994385), (7, 0.020006068), (8, 0.020006327), (9, 0.020005994)]
 

In [ ]:
from gensim import similarities
 
lda_index = similarities.MatrixSimilarity(lda_model[corpus])
 
# Let's perform some queries
similarities = lda_index[lda_model[bow]]
# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda item: -item[1])
 
# Top most similar documents:
print(similarities[:10])
# [(104, 0.87591344), (178, 0.86124849), (31, 0.8604598), (77, 0.84932965), (85, 0.84843522), (135, 0.84421808), (215, 0.84184396), (353, 0.84038532), (254, 0.83498049), (13, 0.82832891)]
 
# Let's see what's the most similar document
document_id, similarity = similarities[0]
print(data[document_id])
 

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "The economy is working better than ever"
x = lda_model.transform(vectorizer.transform([text]))[0]
print(x, x.sum())

In [ ]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

---------

topic detection with scikitlearn

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
 
# Let's see how the first document in the corpus looks like in different topic spaces
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])

In [ ]:

import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

In [ ]:
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(data))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)
 

In [ ]:
svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vectorized.T)
 
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "The economy is working better than ever"
x = lda_model.transform(vectorizer.transform([text]))[0]
print(x, x.sum())

In [ ]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel
 

# putting w2v in motion!


https://ahmedbesbes.com/sentiment-analysis-on-twitter-using-word2vec-and-keras.html

In [1]:
import gensim
import pandas as pd
import pyodbc 
import configLocal

In [2]:
server = configLocal.server
database = configLocal.database
username = configLocal.username
password = configLocal.password
cnxn = configLocal.cnxn
cursor = cnxn.cursor()

In [7]:
modelo = pd.read_csv('./data/WORD2VEC-Twitter-Espa_ol_para_Latinoam_rica__Espa_a_y_Estados_Unidos.csv', index_col="RANK")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
modelo.shape

(159747, 101)

In [8]:
modelo.head()

,WORD,DIMENSION_1,DIMENSION_2,DIMENSION_3,DIMENSION_4,DIMENSION_5,DIMENSION_6,DIMENSION_7,DIMENSION_8,DIMENSION_9,...,DIMENSION_91,DIMENSION_92,DIMENSION_93,DIMENSION_94,DIMENSION_95,DIMENSION_96,DIMENSION_97,DIMENSION_98,DIMENSION_99,DIMENSION_100
RANK,,,,,,,,,,,,,,,,,,,,,
1,de,-1.641680,1.447671,-2.28322,-1.965226,-0.222943,5.105217,-0.120701,-0.126822,-3.177338,...,0.399476,0.887398,-1.994891,2.912205,3.257981,-2.599953,-1.995134,-0.176465,3.946902,-2.792494
2,que,2.167917,-2.581055,-3.29004,-2.788440,1.930902,-2.482932,-2.540504,0.918257,-0.375576,...,2.166590,0.142203,1.698710,-1.105366,6.916267,-3.977985,3.989532,-1.923247,2.035671,2.539047
3,la,0.727598,-3.604198,0.793098,-4.076829,-0.050293,8.435444,-2.233952,2.568616,-2.221771,...,-2.224793,6.436301,-5.083917,-3.572159,3.077797,1.510428,0.011997,2.240481,0.825278,-5.749021
4,a,4.178944,1.255945,-0.714395,0.245530,-4.155548,1.866886,0.096566,1.327916,2.054787,...,3.369879,3.655371,-1.750744,2.508173,6.827713,-5.676323,1.920513,-1.108223,3.379427,-1.197493
5,y,0.799879,-0.130996,-1.71122,-0.395188,-0.195817,2.218104,-0.453679,2.695769,-4.302141,...,1.321977,3.195324,-3.616187,0.961742,4.989526,-4.719704,0.692478,-1.918830,2.808852,0.732657


In [81]:
pd.DataFrame(modelo.iloc[-4:-1])

,WORD,DIMENSION_1,DIMENSION_2,DIMENSION_3,DIMENSION_4,DIMENSION_5,DIMENSION_6,DIMENSION_7,DIMENSION_8,DIMENSION_9,...,DIMENSION_91,DIMENSION_92,DIMENSION_93,DIMENSION_94,DIMENSION_95,DIMENSION_96,DIMENSION_97,DIMENSION_98,DIMENSION_99,DIMENSION_100
RANK,,,,,,,,,,,,,,,,,,,,,
"""error"" : true",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""message"" : ""Internal error""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""status"" : 500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
import numpy as np

In [37]:
a = []
for i in range(modelo.shape[0]):
    a.append(modelo.iloc[i])

In [45]:
matrix_mod = np.array(a)

-----------